In [22]:

# Extencion para conectar a telegram
!pip install pyTelegramBotAPI


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
# Extencion para poder mostrar imagenes como respuestas 
!pip install humingbird

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:

# Utilizamos diccionaios para guardar las respuestas 
#estos son los mensajes que se enviaran al usuario

envios = { "saludo":"Hola, soy un asistente de SIC! gracias por comunicarse con nosotros. ¿En que te puedo ayudar?",
          # avances 
          
           "acerca":"Nuestra compañía ofrece el servicio de envíos internacionales haciendo posible que sus documentos, paquetes y mercancías lleguen a más de 220 países y territorios en los 5 continentes,\n"
                    "prestándole adicionalmente el servicio de asesoría que le permita hacer más fácil el proceso de su envío."
                    "Mediante este chat podrás saber toda la información que necesites o te permitiremos contactar alguno de nuestros agentes de la empresa.",
          
          "Recibo":"Contamos con infraestructuras propias y sistematizadas en todos nuestros puntos de recibo, garantizando confiabilidad, seguridad y asesoría especializada."
                    "Nuestro servicio de recogidas y entregas de lunes a sábado a domicilio, con frecuencia de vuelo diaria para sus exportaciones. "
                    "Recogemos sin costo adicional en el territorio nacional.",
          
          "Envio":"-El máximo valor declarado para exportaciones de tráfico postal y envíos urgentes es de USD 5.000. Para exportación tradicional máximo valor declarado debe ser menor a U$ 50.000 *para mayor valor llamar a asesoría.\n"
                    "-El máximo Peso/Volumen permitido es 70 kilogramos (por unidad de empaque).\n"
                    "-Los cargos por el peso dimensional de los envíos se determinan de acuerdo a las normas estándar de volumen vigentes de la Asociación de Transporte Aéreo (IATA), las cuales están sujetas a cambio sin previo aviso.\n"
                    "-No se hacen entregas a Apartados de Correos, P.O Box y/o apartados aéreos. Es necesario indicar una dirección física correcta.\n"
                    "-ESTA PROHIBIDO EL ENVÍO DE:\n" 
                    "-Animales \n"
                    "-Armas de fuego, municiones, explosivos\n"
                    "-Artículos ilegales\n",
          
          "Nece":"Todos nuestros envíos necesitan estos documentos: guía aérea, rótulo o etiqueta, factura comercial, carta de responsabilidad y documentos requeridos por el país destino.",

          "Tarifa":"Imagen",#aqui va una imagen 

          "Cita": "Para mayor información contacar algunos de nuestro agentes llamando al:" 
                  "253-0000 o 254-0000",
          "Despedida": "Le agradecemos por usar nuestro servicios. si tiene alguna duda adicional puede escribirnos con total confianza fue un placer serviles !!hasta la próxima¡¡",
          }

imagenes_costo= {
    "costos":[" /content/drive/MyDrive/Imagen de WhatsApp 2023-01-24 a las 10.35.10.jpg"]
}
                

In [25]:

# Prueba de funcionalidad 
#Este algoritmo nos permite recibir preguntas del usuario, entender la intencion de la pregunta y enviar una respuesta 
#NO CONTIENE MACHINE LEARNING 
# importamos las librerias de re y ramdom para...

import re
import random
import humingbird

# # cramos una funcion donde retorne las exepciones cuando la probabilidad sea menor a 1
def unknown():
    response = ['Disculpa no te estoy entendiendo', 'Puedes repetirlo otra vez']
    return response[random.randrange(2)]

#creamos otra funcion donde revisamos el texto
def get_response(txt):
    split_message = re.split(r'\s|[:;.?!-_]\s*f',txt.lower())
    response = check_all_message(split_message)
    return response


# En esta funcion calculamos la probabilidad de respustas segun el texto
def message_probability(user_msn, recognized_word, single_response=False,required_word = []):
    message_certainly = 0
    has_required_word = True
    print(f'user msn {user_msn}')


    #empezamos a a iterar sumando las palabras reconocidas y vamos sumando 1 
    for word in user_msn:
        if word in recognized_word:
            message_certainly+=1
      # por aqui calculamos la seguridad segun la probabilidad del mensaje      
    percentage = float(message_certainly)/float(len(recognized_word))
    for word in required_word:
        if word not in user_msn:
            has_required_word = False
            break
    if has_required_word or single_response:
        return float(percentage * 100)
    else:
        return 0

#Funcion en donde revisamos todo los mensajes de todas las posibles respuetas 
def check_all_message(txt):
    
    highest_prob = {}    
    
    def response(bot_response,list_of_words,single_response = False,required_words = []):
        nonlocal highest_prob
        highest_prob[bot_response]= message_probability(txt,list_of_words,single_response,required_words)
    
    #arreglar error 

    #Por aca estan las reglas que para poder imprimir las respuetas correcta 
    v_response = ['estoy','muy','bien']
                
    response(envios["saludo"],["hola","buenas","hi","hello"],single_response=True)
    response(envios["acerca"],["esta","que","servicios","servicio","ofrece","esta","empresa"],required_words=["servicios"])
    response(envios["Recibo"],["me","puede","dar","informacion","sobre","las","entregas","recibo","costo"],required_words=["entregas"])
    response(envios["Nece"],["que","documento","necesito","documentacion","regla","una","consulta"],required_words=["documentos"])
    response(envios["Despedida"],["muchas", "Gracias" , "Thank you" ," les","agradesco","Adios" ,"bye"],single_response=True)
    response(envios["Envio"],["informacion","sobre","los","envios","envio"],required_words=["información"])
    response('¡Excelente! ¿ Que deseas ?',v_response,required_words=['estoy'])


     #response(envios["saludos"],v_response,required_words=["hi","hola",])



    #segun el mayor indice de probabilidad imprimira la respuesta
    best_match = max(highest_prob,key=highest_prob.get)
    print(highest_prob)
    # si en tal caso no se encunetra contesto de la pregunta el chatbot retoranara a la funcion unknown
    return unknown() if highest_prob[best_match] < 1 else best_match
    


In [26]:
#codigo para la coneccion con telegram

import telebot 
token = "5851734942:AAGfKcGcHpunyOBPhT2WBzz7JUB-Rp0YidM" 
BOT_TOKEN = token 
bot = telebot.TeleBot(BOT_TOKEN)



@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(func=lambda m: True)
def echo_all(message):
    response = get_response(message.text)
    bot.reply_to(message, response)
    print(message)
   
   


In [ ]:
bot.infinity_polling()

user msn ['hola']
user msn ['hola']
user msn ['hola']
user msn ['hola']
user msn ['hola']
user msn ['hola']
user msn ['hola']
{'Hola, soy un asistente de SIC! gracias por comunicarse con nosotros. ¿En que te puedo ayudar?': 25.0, 'Nuestra compañía ofrece el servicio de envíos internacionales haciendo posible que sus documentos, paquetes y mercancías lleguen a más de 220 países y territorios en los 5 continentes,\nprestándole adicionalmente el servicio de asesoría que le permita hacer más fácil el proceso de su envío.Mediante este chat podrás saber toda la información que necesites o te permitiremos contactar alguno de nuestros agentes de la empresa.': 0, 'Contamos con infraestructuras propias y sistematizadas en todos nuestros puntos de recibo, garantizando confiabilidad, seguridad y asesoría especializada.Nuestro servicio de recogidas y entregas de lunes a sábado a domicilio, con frecuencia de vuelo diaria para sus exportaciones. Recogemos sin costo adicional en el territorio nacional